# **SOM** - Grid map & Sampled initialization of weights

## Import libraries

In [ ]:
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot as plt
import time
from math import exp


## Load the dataset

In [ ]:
iterations = 20
neighbourhood_radius = 1

(_, _), (test_images, test_labels) = mnist.load_data()

reduced_mnist_images = [test_images[np.random.choice(a=np.where(test_labels == j)[0], size=100)] for j in range(10)]
reduced_mnist_images = np.array(reduced_mnist_images, dtype='float32') / 255

reduced_mnist_images = reduced_mnist_images.reshape((1000, 784))
reduced_mnist_labels = np.array([100*[i] for i in range(10)], dtype='int').ravel()

shuffled_indices = np.random.permutation(np.arange(1000))

reduced_mnist_images = reduced_mnist_images[shuffled_indices]
reduced_mnist_labels = reduced_mnist_labels[shuffled_indices]

## Implementation

In [ ]:
def learning_rate_generator_with_decay():
   global iterations
   time = 0
   while True:
       yield (0.5 - (time/(iterations*2)))
       time += 1

In [ ]:
# Define and fill the weights of grid network with random numbers (between 0 and 1)

grid_som_weight_matrix = reduced_mnist_images[np.random.choice(np.arange(1000), size=625, replace=False)]
grid_som_weight_matrix = grid_som_weight_matrix.reshape((25, 25, 784))

In [ ]:
lr_with_decay = learning_rate_generator_with_decay()    # instantiate a lr generator
weights_snapshot = {0 : grid_som_weight_matrix.copy()}   # snapshot for some epochs
coverage_history = np.zeros((iterations+1, 25, 25), dtype=bool)

winning_counts = dict()
winning_classes_counts = dict()

for epoch in range(iterations):

    lr = next(lr_with_decay)    # Get new learning rate value

    start = time.time()     # start the timer

    for x in range(1000):

        # Calculate Euclidean distance for all neurons
        distance_matrix = np.sqrt(((grid_som_weight_matrix - reduced_mnist_images[x])**2).sum(axis = 2))

        # Find index of minimum distance
        winner_unit = np.argmin(distance_matrix)
        d0_winner = winner_unit // 25
        d1_winner = winner_unit % 25
        coverage_history[epoch, d0_winner, d1_winner] = True


        for d0 in range(d0_winner - 1, d0_winner + 1 + 1):
            for d1 in range(d1_winner - 1, d1_winner + 1 + 1):
                if 0 <= d0 <= 24 and 0 <= d1 <= 24:
                    if (d0 == d0_winner and d1 == d1_winner) or neighbourhood_radius == 1:
                        dif = reduced_mnist_images[x] - grid_som_weight_matrix[d0][d1]
                        grid_som_weight_matrix[d0][d1] += (dif * lr)

        # Decrease the neighbourhood radius if we are in the middle of running
        if round(iterations/3) == epoch:
            neighbourhood_radius = 0
        
        # If it is the last epoch, we want to store the winning statistics of neurons
        if epoch + 1 == iterations:
            winning_counts[(d0_winner,d1_winner)] = winning_counts.get((d0_winner,d1_winner), 0) + 1
            if winning_classes_counts.get((d0_winner,d1_winner), None) == None:
                winning_classes_counts[(d0_winner,d1_winner)] = dict()
            winning_classes_counts[(d0_winner,d1_winner)][reduced_mnist_labels[x]] = winning_classes_counts[(d0_winner,d1_winner)].get(reduced_mnist_labels[x], 0) + 1
    
    end = time.time()
    print("epoch %d - %.3f s" % (epoch + 1, end - start))

    weights_snapshot[epoch + 1] = grid_som_weight_matrix.copy()


epoch 1 - 0.976 s
epoch 2 - 0.982 s
epoch 3 - 1.016 s
epoch 4 - 1.062 s
epoch 5 - 1.193 s
epoch 6 - 1.263 s
epoch 7 - 1.390 s
epoch 8 - 1.377 s
epoch 9 - 1.407 s
epoch 10 - 1.447 s
epoch 11 - 1.443 s
epoch 12 - 1.316 s
epoch 13 - 1.313 s
epoch 14 - 1.352 s
epoch 15 - 1.366 s
epoch 16 - 1.366 s
epoch 17 - 1.391 s
epoch 18 - 1.393 s
epoch 19 - 1.395 s
epoch 20 - 1.502 s


### Show twenty neurons with the most winnings at the last epoch

In [ ]:
twenty_neurons_with_most_winning = [k for k, v in sorted(winning_counts.items(), key=lambda item: item[1], reverse = True)][:20]

stat_table = np.zeros((20,10), dtype='int')

for i, neu in enumerate(twenty_neurons_with_most_winning):
    for class_label in winning_classes_counts[neu].keys():
        stat_table[i][class_label] = winning_classes_counts[neu][class_label]

import pandas as pd 

df = pd.DataFrame(stat_table, index=twenty_neurons_with_most_winning, columns=['label {0}'.format(i) for i in range(0,10)])
df['Total'] = df.sum(axis=1)
print("Total sum:  %d images" % df['Total'].sum())
df

Total sum:  115 images


,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,label 9,Total
"(11, 11)",0,8,0,0,0,0,0,0,0,0,8
"(9, 16)",0,7,0,0,0,0,0,0,0,0,7
"(7, 15)",0,0,7,0,0,0,0,0,0,0,7
"(12, 12)",0,6,0,0,0,0,0,0,0,0,6
"(8, 11)",0,0,0,0,0,0,0,6,0,0,6
"(2, 17)",5,0,0,0,0,0,0,0,0,1,6
"(18, 5)",0,6,0,0,0,0,0,0,0,0,6
"(20, 12)",0,0,0,0,0,0,6,0,0,0,6
"(12, 21)",0,0,0,0,6,0,0,0,0,0,6
"(2, 6)",0,0,0,0,0,0,0,6,0,0,6


### Generate and save figures

In [ ]:
import os
os.makedirs('./grid-topology_sampled-weights/', exist_ok=True)

In [ ]:
for epoch, weight_mat in weights_snapshot.items():
    fig = plt.figure(figsize=(15,15))
    axes = [fig.add_subplot(25,25,i+1) for i in range(625)]

    for n, ax in enumerate(axes):
        d0 = n // 25
        d1 = n % 25

        ax.imshow(weight_mat[d0,d1].reshape((28,28)), cmap=("gray_r" if coverage_history[epoch, d0, d1] == False else "plasma"))

        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        if n >= 600: ax.set_xlabel(n%25 + 1)
        if n%25==0: ax.set_ylabel(n//25 + 1)
        ax.tick_params(axis='both', which='both', length=0)
        ax.label_outer

    fig.subplots_adjust(wspace=0, hspace=0)
    fig.suptitle("Epoch %d - [Grid Topology , Sampled Weights Initialization]" % epoch, fontsize=18)

    # write the figure for this epoch on disk
    path = f"./grid-topology_sampled-weights/epoch-{epoch}"
    fig.savefig(path)
    plt.close(fig)

    print(f"Saved: {path}.png")

Saved: ./grid-topology_sampled-weights/epoch-0.png
Saved: ./grid-topology_sampled-weights/epoch-1.png
Saved: ./grid-topology_sampled-weights/epoch-2.png
Saved: ./grid-topology_sampled-weights/epoch-3.png
Saved: ./grid-topology_sampled-weights/epoch-4.png
Saved: ./grid-topology_sampled-weights/epoch-5.png
Saved: ./grid-topology_sampled-weights/epoch-6.png
Saved: ./grid-topology_sampled-weights/epoch-7.png
Saved: ./grid-topology_sampled-weights/epoch-8.png
Saved: ./grid-topology_sampled-weights/epoch-9.png
Saved: ./grid-topology_sampled-weights/epoch-10.png
Saved: ./grid-topology_sampled-weights/epoch-11.png
Saved: ./grid-topology_sampled-weights/epoch-12.png
Saved: ./grid-topology_sampled-weights/epoch-13.png
Saved: ./grid-topology_sampled-weights/epoch-14.png
Saved: ./grid-topology_sampled-weights/epoch-15.png
Saved: ./grid-topology_sampled-weights/epoch-16.png
Saved: ./grid-topology_sampled-weights/epoch-17.png
Saved: ./grid-topology_sampled-weights/epoch-18.png
Saved: ./grid-topology

In [ ]:
!zip -r grid_sampled.zip /content/grid-topology_sampled-weights

  adding: content/grid-topology_sampled-weights/ (stored 0%)
  adding: content/grid-topology_sampled-weights/epoch-20.png (deflated 10%)
  adding: content/grid-topology_sampled-weights/epoch-14.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-13.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-3.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-5.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-4.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-10.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-6.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-2.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-0.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-18.png (deflated 1%)
  adding: content/grid-topology_sampled-weights/epoch-11.png (deflated 1%)
  adding: content/grid-topology_sampled-weig